In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install pyspark

In [ ]:
# Importing the Package
from pyspark import SparkContext
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

print("All Package Loaded")

In [ ]:
#Load the Data
spark=SparkSession.builder.master('local').appName("Classification Heart Failure").getOrCreate()
data=spark.read.csv("../input/heart-failure-clinical-data", inferSchema=True, header=True)
data.show(truncate=False, n=5)

In [ ]:
#Cek the data
data.printSchema()

In [ ]:
#Split data
data_split=data.randomSplit([0.7,0.3])
train=data_split[0]
test=data_split[1]
print("train data:\n", train.count(), "\ntest data:\n", test.count())

In [ ]:
#Featuring Field
train.show(2)
assembler=VectorAssembler(inputCols=['age','anaemia','creatinine_phosphokinase',
                                     'diabetes','ejection_fraction','high_blood_pressure',
                                     'platelets','serum_creatinine','serum_sodium',
                                     'sex','smoking','time'],outputCol='features')
trainingdatafinal=assembler.transform(train).select(col('features'),col('DEATH_EVENT').alias('label'))
trainingdatafinal.show(truncate=False, n=5)

In [ ]:
#Training Model
Classification=LogisticRegression(featuresCol='features', labelCol='label', maxIter=100,regParam=0.3)
model=Classification.fit(trainingdatafinal)
print("Training Done")

In [ ]:
#Preparing data test
testingdatafinal=assembler.transform(test).select(col('features'),
                                                  col('DEATH_EVENT').alias("label"))
testingdatafinal.show(truncate=False, n=5)

In [ ]:
#Testing model using data test
raw_prediction=model.transform(testingdatafinal)
raw_prediction.show(truncate=False, n=4)
raw_prediction.printSchema()


In [ ]:
#Final test look
final_prediction=raw_prediction.select('features','label','prediction')
final_prediction.show(truncate=False, n=5)

In [ ]:
#Accuracy
Total_true=final_prediction.filter(final_prediction['label']==final_prediction['prediction']).count()
Total_data=final_prediction.count()
print("Accuracy is :\n", Total_true/Total_data*100,"%")

#I have using some methods, in non spark analysis, wait me to  improve this accuracy. Please upvote! Thank you.